In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, regexp_replace, expr, split, from_json
from pyspark.sql.types import ArrayType, IntegerType, StringType, StructType, StructField

# import mysql.connector
from datetime import datetime

import pandas as pd
import pymysql
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# MySQL 데이터베이스에 연결

# MySQL JDBC 드라이버 사용을 위한 설정
hostname = "vowing-database-1.cluster-cuyfo2igvloh.ap-northeast-2.rds.amazonaws.com"      # MySQL 호스트명
port = "3306"                   # MySQL 포트 (기본값은 3306)
database = "vowing"       # 데이터베이스 이름
username = "vowing"             # 사용자 이름
password = "vowing1234!@"       # 비밀번호

jdbc_url = f"jdbc:mysql://{hostname}:{port}/{database}?autoReconnect=true&connectTimeout=60000"
connection_properties = {
  "user" : username,
  "password" : password,
  "driver" : "com.mysql.cj.jdbc.Driver"
}

In [9]:
# 가상 환경의 Python 경로를 지정
python_path = "C:/Users/admin/anaconda3/envs/my_conda_01/python.exe"

# Spark 세션 생성
spark = SparkSession.builder \
    .appName("voice_metadata integrating operation") \
    .config("spark.driver.memory", "9g") \
    .config("spark.executor.memory", "9g") \
    .config("spark.driver.maxResultSize", "5g") \
    .config("spark.pyspark.python", python_path) \
    .config("spark.local.dir", "D:/spark_tmp") \
    .config("spark.jars", "C:/Program Files/mysql-connector-j-8.2.0/mysql-connector-j-8.2.0.jar") \
    .getOrCreate()

# SparkContext 가져오기
sc = spark.sparkContext

# Spark 중간 파일이 저장되는 경로 확인
current_spark_local_dir = spark.conf.get("spark.local.dir", "Not Set")
print("Spark local directory:", current_spark_local_dir)

Spark local directory: D:/spark_tmp


In [4]:
# # 데이터베이스 연결 설정
# conn = pymysql.connect(
#     host="vowing-database-1.cluster-cuyfo2igvloh.ap-northeast-2.rds.amazonaws.com",
#     user='vowing',
#     password='vowing1234!@',
#     db='vowing'
# )

# # 테이블 데이터를 pandas DataFrame으로 로드
# def load_table(table_name):
#     sql_query = f"SELECT * FROM {table_name}"
#     df = pd.read_sql_query(sql_query, conn)
#     return df

# # '포인트 벌기' 관련 테이블 로드 및 로컬에 Parquet 파일로 저장
# ad_df = load_table("ad")
# ad_table = pa.Table.from_pandas(ad_df)
# pq.write_table(ad_table, 'D:/ad.parquet')

# ad_participation_df = load_table("ad_participation")
# ad_participation_table = pa.Table.from_pandas(ad_participation_df)
# pq.write_table(ad_participation_table, 'D:/ad_participation.parquet')

# # '암기플러스' 관련 테이블 로드 및 로컬에 Parquet 파일로 저장
# m10n_df = load_table("m10n")
# m10n_table = pa.Table.from_pandas(m10n_df)
# pq.write_table(m10n_table, 'D:/m10n.parquet')

# m10n_participation_df = load_table("m10n_participation")
# m10n_participation_table = pa.Table.from_pandas(m10n_participation_df)
# pq.write_table(m10n_participation_table, 'D:/m10n_participation.parquet')

# # 데이터베이스 연결 종료
# conn.close()

In [10]:
# '포인트 벌기' 관련 테이블들을 로드
table_name = "ad"
ad_df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)
table_name = "ad_participation"
ad_participation_df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)

# '암기플러스' 관련 테이블들을 로드
table_name = "m10n"
m10n_df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)
table_name = "m10n_participation"
m10n_participation_df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)

Py4JJavaError: An error occurred while calling o46.jdbc.
: java.lang.ClassNotFoundException: com.mysql.cj.jdbc.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:587)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:520)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:41)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationPro